In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv

In [13]:
# Clean dataset, take only necessary columns, create new dataframes and save as csv

# File to Load
fastfood_to_load = "datasets/fast-food-restaurants/Datafiniti_Fast_Food_Restaurants.csv"
census_to_load = "datasets/us-census-demographic-data/acs2017_census_tract_data.csv"

# Read the CSV files
fastfood = pd.read_csv(fastfood_to_load)
census = pd.read_csv(census_to_load)

# Create FastFood dataset
ff_id = fastfood['id']
ff_name = fastfood['name']
ff_state = fastfood['province']
dict1 = {'state': ff_state, 'name': ff_name}  
fastfood_df = pd.DataFrame(dict1) 

# Output File (CSV)
output_ff_file = "output_data/fastfood.csv"

# Write DF to a CSV
fastfood_df.to_csv(output_ff_file, sep=';', encoding='utf-8')


# Create Census dataset
#Columns: State, Income, Professional, Unemployment
us_id = census['TractId']
us_state = census['State']
us_income = census['Income']
us_professional = census['Professional']
us_unemployment = census['Unemployment']
dict2 = {'state': us_state, 'income': us_income, 'professional': us_professional, 'unemployment': us_unemployment}
census_df = pd.DataFrame(dict2)

# Output File (CSV)
output_us_file = "output_data/census.csv"

# Write output to the file
census_df.to_csv(output_us_file, sep=';', encoding='utf-8')


In [14]:
fastfood_df.head()

,state,name
0,LA,SONIC Drive In
1,LA,SONIC Drive In
2,TN,Taco Bell
3,TN,Arby's
4,GA,Steak 'n Shake


In [15]:
census_df.head()

,state,income,professional,unemployment
0,Alabama,67826.0,38.5,4.6
1,Alabama,41287.0,30.5,3.4
2,Alabama,46806.0,27.9,4.7
3,Alabama,55895.0,29.0,6.1
4,Alabama,68143.0,48.8,2.3


## Questions:
1. most popular FFR by state
2. correlation between white collar jobs and FFR
3. correlation between unemployment rate and FFR
4. correlation between income and FFR

In [16]:
#ffbs_df most popular fastfood in each state. 
ffbs_df = fastfood_df.groupby('state', as_index=False).agg({'name': lambda x: x.value_counts().index[0]})

In [17]:
ffbs_df.head()

,state,name
0,AK,Subway
1,AL,Taco Bell
2,AR,McDonald's
3,AZ,McDonald's
4,CA,McDonald's


In [18]:
census_df.groupby('state', as_index=False).mean()

,state,income,professional,unemployment
0,Alabama,45938.212947,30.782298,8.651574
1,Alaska,73796.757576,35.335542,8.648795
2,Arizona,57815.571807,34.574455,7.598416
3,Arkansas,44245.267936,30.726131,7.039708
4,California,73070.965821,37.051064,7.874869
5,Colorado,69595.696429,40.284559,5.490137
6,Connecticut,79622.418919,41.552842,7.749819
7,Delaware,64310.537383,39.043925,6.601869
8,District of Columbia,82936.299435,56.355866,9.784916
9,Florida,54263.602892,34.220504,7.442737
